In [1]:
from openai.embeddings_utils import cosine_similarity
from evalio.semantics import visualize_embeddings, get_embedding, get_nearest_neighbors
from evalio.util import convert_string_to_numpy_array
import pandas as pd

from sklearn.metrics import classification_report, PrecisionRecallDisplay

In [ ]:
EMBEDDING_COLUMN_NAME = 'embedding'
SCORE_COLUMN_NAME = 'Score'

df = pd.read_csv('data/fine_food_reviews_with_embeddings_1k.csv')

# Visualize embeddings

In [ ]:
visualize_embeddings(df, embeddings_column='embedding', score_column='Score')

# 2-class Precision-Recall curve (review sentiment)

In [ ]:
df['embedding'] = convert_string_to_numpy_array(df['embedding'])

# convert 5-star rating to binary sentiment
df = df[df.Score != 3]
df["sentiment"] = df.Score.replace({1: "negative", 2: "negative", 4: "positive", 5: "positive"})

In [ ]:
def evaluate_embeddings_approach(labels=('negative', 'positive')):
    label_embeddings_ = [get_embedding(label) for label in labels]

    def label_score(review_embedding, label_embeddings):
        return cosine_similarity(review_embedding, label_embeddings[1]) - cosine_similarity(
            review_embedding, label_embeddings[0]
        )

    probas = df["embedding"].apply(lambda x: label_score(x, label_embeddings_))
    preds = probas.apply(lambda x: 'positive' if x > 0 else 'negative')

    report = classification_report(df.sentiment, preds)
    print(report)

    display = PrecisionRecallDisplay.from_predictions(df.sentiment, probas, pos_label='positive')
    _ = display.ax_.set_title("2-class Precision-Recall curve")

In [ ]:
evaluate_embeddings_approach(labels=['negative', 'positive'])

In [ ]:
evaluate_embeddings_approach(
    labels=['An Amazon review with a negative sentiment.', 'An Amazon review with a positive sentiment.']
)

# Clustering

In [ ]:
from evalio.semantics import create_clusters_from_embeddings, create_clusters_from_column

CLUSTER_COLUMN_NAME = 'Cluster'
N_CLUSTERS = 4

df['Cluster'] = create_clusters_from_column(df['embedding'], n_clusters=N_CLUSTERS)
# create_clusters_from_embeddings(
#     df,
#     embeddings_column=EMBEDDING_COLUMN_NAME,
#     target_cluster_column=CLUSTER_COLUMN_NAME,
#     n_clusters=N_CLUSTERS,
# )

In [ ]:
from evalio.semantics import visualize_embedding_clusters

visualize_embedding_clusters(
    df, embeddings_column='embedding', target_cluster_column='Cluster', score_column='Score', n_clusters=N_CLUSTERS
)

In [ ]:
from evalio.semantics import classify_clusters_with_llm

classify_clusters_with_llm(df, cluster_column='Cluster', n_clusters=N_CLUSTERS)

# Semantic search

In [ ]:
from evalio.semantics import search_reviews


results = search_reviews(df, "delicious beans", embeddings_column='embedding', top_n=3)

In [17]:
news_df = pd.read_csv('data/AG_news_samples.csv')
for idx, row in news_df.head(3).iterrows():
    print("")
    print(f"Title: {row['title']}")
    print(f"Description: {row['description']}")
    print(f"Label: {row['label']}")


Title: World Briefings
Description: BRITAIN: BLAIR WARNS OF CLIMATE THREAT Prime Minister Tony Blair urged the international community to consider global warming a dire threat and agree on a plan of action to curb the  quot;alarming quot; growth of greenhouse gases.
Label: World

Title: Nvidia Puts a Firewall on a Motherboard (PC World)
Description: PC World - Upcoming chip set will include built-in security features for your PC.
Label: Sci/Tech

Title: Olympic joy in Greek, Chinese press
Description: Newspapers in Greece reflect a mixture of exhilaration that the Athens Olympics proved successful, and relief that they passed off without any major setback.
Label: Sports


In [18]:
news_descriptions = news_df["description"].tolist()
news_descriptions

['BRITAIN: BLAIR WARNS OF CLIMATE THREAT Prime Minister Tony Blair urged the international community to consider global warming a dire threat and agree on a plan of action to curb the  quot;alarming quot; growth of greenhouse gases.',
 'PC World - Upcoming chip set will include built-in security features for your PC.',
 'Newspapers in Greece reflect a mixture of exhilaration that the Athens Olympics proved successful, and relief that they passed off without any major setback.',
 'SAN JOSE, Calif. -- Apple Computer (Quote, Chart) unveiled a batch of new iPods, iTunes software and promos designed to keep it atop the heap of digital music players.',
 'Any product, any shape, any size -- manufactured on your desktop! The future is the fabricator. By Bruce Sterling from Wired magazine.',
 'KABUL, Sept 22 (AFP): Three US soldiers were killed and 14 wounded in a series of fierce clashes with suspected Taliban fighters in south and eastern Afghanistan this week, the US military said Wednesday.

In [19]:
news_df["description"].values[0] == news_df["description"][0] == news_descriptions[0]

True

In [20]:
first_news_description_embedding = get_embedding(news_descriptions[0])
first_news_description_embedding

array([-0.01060311, -0.02240512, -0.00866941, ..., -0.00174033,
       -0.03568318, -0.01437382], dtype=float32)

In [21]:
get_nearest_neighbors(news_descriptions, 0, k_nearest_neighbors=5)

KeyboardInterrupt: 

In [23]:
get_embedding.cache_info()

CacheInfo(hits=403, misses=1578, maxsize=1000, currsize=1000)